# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_extraction.text import (
    CountVectorizer
)
from sklearn.feature_selection import (
    SelectKBest, SelectPercentile,
    SelectFpr, VarianceThreshold,
    SelectFromModel,
    chi2, f_classif,
)
from sklearn.ensemble import (
    RandomForestClassifier
)
from sklearn.linear_model import (
    LogisticRegression
)
from sklearn.pipeline import (
    FeatureUnion, Pipeline
)
from scipy.sparse import hstack


%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
display(
    X_train.isnull().sum()[X_train.isnull().sum() > 0],
    X_test.isnull().sum()[X_test.isnull().sum() > 0]
)

funder                3635
installer             3655
subvillage             371
public_meeting        3334
scheme_management     3877
scheme_name          28166
permit                3056
dtype: int64

funder                869
installer             877
subvillage             99
public_meeting        821
scheme_management     969
scheme_name          7092
permit                737
dtype: int64

In [5]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [6]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [7]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

X_train.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_train.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))
X_test.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_test.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))


X_train.loc[:, 'date_recorded_month'] /= 12
X_test.loc[:, 'date_recorded_month'] /= 12

In [8]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [9]:
X_train.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)
X_test.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)

In [10]:
heightscale = StandardScaler().fit(pd.concat((X_train.loc[:, 'gps_height'],
                                              X_test.loc[:, 'gps_height']), ignore_index=True))
X_train.loc[:, 'gps_height'] = heightscale.transform(X_train.loc[:, 'gps_height']).reshape(-1, 1)
X_test.loc[:, 'gps_height'] = heightscale.transform(X_test.loc[:, 'gps_height']).reshape(-1, 1)

/home/god/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-pack

In [11]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population'])
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population'])
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
# display(X_test.loc[:,'amount_per_man'])
amntpmscaler = StandardScaler().fit(X_test.loc[:, 'amount_per_man'])
X_train.loc[:,'amount_per_man'] = amntpmscaler.transform(X_train.loc[:,'amount_per_man']).reshape(-1, 1)
X_test.loc[:,'amount_per_man'] = amntpmscaler.transform(X_test.loc[:,'amount_per_man']).reshape(-1, 1)

/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single 

In [12]:
amounttshscaler = StandardScaler().fit(pd.concat((X_train.loc[:, 'amount_tsh'],
                                              X_test.loc[:, 'amount_tsh']), ignore_index=True))
X_train.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_train.loc[:, 'amount_tsh']).reshape(-1, 1)
X_test.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_test.loc[:, 'amount_tsh']).reshape(-1, 1)

/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single 

In [13]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [14]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [15]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [16]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [17]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [18]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [19]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [20]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [21]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [22]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [23]:
drop_cats = ['source_type', 'source_class',
             'water_quality', 'quality_group',
             'quantity_group', 'waterpoint_type_group',
             'payment_type', 'extraction_type_group',
             'extraction_type_class', 'management_group',
             'construction_year', 'date_recorded_month'
]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [24]:
X_train.iloc[0,:]

amount_tsh                                         1.95453
funder                                               Roman
gps_height                                         1.04558
installer                                            Roman
longitude                                          34.9381
latitude                                          -9.85632
wpt_name                                                 1
basin                                           Lake Nyasa
subvillage                                        Mnyusi B
region                                              Iringa
region_code                                             11
district_code                                            5
lga                                                 Ludewa
ward                                              Mundindi
population                                             109
public_meeting                                           1
scheme_management                                      V

In [25]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []
text_selfeat_names = []

In [26]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [27]:
# lonlat shall be clusterized before usage
# day probably doesn't matter, but month might be (seasonality)
# the years in test and train do not intersect
X_train.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')
X_test.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')

In [28]:
display(X_train.shape, X_test.shape)

(59400, 28)

(14850, 28)

In [29]:
sel = Pipeline([
    ('varthres', VarianceThreshold(5e-5)),
    #('interact', PolynomialFeatures(2, interaction_only=True)),
    #('feats', FeatureUnion([
        # ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        # ('chi2', SelectKBest(chi2, min(128, tr.shape[1])).fit(tr, ytr),)
        # OHE-encoded features are supposed to have additive impact, so linear model:
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=100, max_depth=12)))
    #])),
])
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = CountVectorizer(ngram_range=(1,2), binary=True)
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)

    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    for i in range(tr.shape[1]):
        text_selfeat_names.append('%s_%s' % (cat, i))
    #tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    #te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    if tr.shape[1] > 0:
        text_feats_train.append(tr)
        text_feats_test.append(te)

text_feats_train = hstack(text_feats_train) # pd.DataFrame(hstack(text_feats_train), columns=text_selfeat_names)
text_feats_test = hstack(text_feats_test) # pd.DataFrame(hstack(text_feats_test), columns=text_selfeat_names)
# print(sel.fit_transform(text_feats_train, ytr).toarray().shape)
# sel_text_feats = SelectKBest(chi2, 48).fit(text_feats_train, ytr) # SelectFpr(f_classif, 1e-19)
sel_text_feats = sel
text_feats_train = sel.fit_transform(text_feats_train, ytr)
text_feats_test = sel.transform(text_feats_test)
print('cat features shape: ', text_feats_train.shape)
# print('top catfeats: ', sel.named_steps['linear'].estimator_.coef_[:20] )
del tr, te
X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore'),
                     pd.DataFrame(text_feats_train.toarray())],
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore'),
                    pd.DataFrame(text_feats_test.toarray())],
                    axis=1)

cat features shape:  (59400, 129)


In [30]:
X_train.head()

,amount_tsh,gps_height,wpt_name,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,...,119,120,121,122,123,124,125,126,127,128
0,1.95452941861,1.04558069781,1,109,1.0,0.0,1,0,12,0.031485078672,...,1,1,1,0,1,0,0,0,1,0
1,-0.109636705889,1.05857227813,1,280,0.5,1.0,1,0,3,-0.0940907887382,...,0,0,0,1,1,0,0,0,1,0
2,-0.101036013704,0.0293504149544,1,250,1.0,1.0,1,0,4,-0.0938626592458,...,0,0,0,1,1,0,0,0,0,0
3,-0.109636705889,-0.581253860116,1,58,1.0,1.0,1,0,27,-0.0940907887382,...,0,0,0,0,1,0,0,0,0,0
4,-0.109636705889,-0.960896707264,1,0,1.0,1.0,0,0,2011,-0.0940907887382,...,0,0,0,1,1,0,0,0,1,0


In [31]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('dry_season', dtype('O')),
 ('rainy_season', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [32]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
                      
#y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
#display(
#    np.where(ytr != 0)[0][:40],
#    np.where(ycv != 0)[0][:40],
#    np.where(yva != 0)[0][:40],)
#ytr = y_bin.transform(ytr)
#ycv = y_bin.transform(ycv)
#yva = y_bin.transform(yva)

In [33]:
def y_to_df(y, ids):
    #y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [34]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'dry_season',
 'rainy_season',
 'age',
 'amount_per_man',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128]

In [35]:
display(xtr.shape, xte.shape)

(41578, 139)

(14850, 139)

In [36]:
# cat_features = np.array([xtr.columns.get_loc(c) for c in text_features if c in xtr.columns])
cat_features = []
sel = Pipeline([
    ('varthres', VarianceThreshold(1e-5)),
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=300), '0.8*mean'))
])
xtr = sel.fit_transform(xtr, ytr)
xcv = sel.transform(xcv)
xva = sel.transform(xva)
xte = sel.transform(xte)

display(xtr.shape,
        sel.steps[-1][-1].estimator_.feature_importances_)

In [37]:
#X_train.to_csv('X_train_prepr2.csv')
#X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
#pd.Series(ytr).to_csv('y_train_prepr2.csv')
#pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [38]:
import catboost

In [39]:
clf = catboost.CatBoostClassifier(2500, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)

In [40]:
clf.fit(xtr, ytr, cat_features=cat_features,
        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.6939487229	test 0.6925235743	bestTest 0.6925235743	passed: 0.214 sec	total: 417ms	remaining: 17m 21s
1:	learn 0.6939487229	test 0.6925235743	bestTest 0.6925235743	passed: 0.191 sec	total: 608ms	remaining: 12m 39s
2:	learn 0.7043628842	test 0.703973956	bestTest 0.703973956	passed: 0.202 sec	total: 810ms	remaining: 11m 14s
3:	learn 0.6958006638	test 0.6949932645	bestTest 0.703973956	passed: 0.204 sec	total: 1.01s	remaining: 10m 32s
4:	learn 0.6939487229	test 0.6925235743	bestTest 0.703973956	passed: 0.187 sec	total: 1.2s	remaining: 9m 59s
5:	learn 0.6939487229	test 0.6925235743	bestTest 0.703973956	passed: 0.199 sec	total: 1.4s	remaining: 9m 41s
6:	learn 0.6939487229	test 0.6925235743	bestTest 0.703973956	passed: 0.18 sec	total: 1.58s	remaining: 9m 22s
7:	learn 0.6939487229	test 0.6925235743	bestTest 0.703973956	passed: 0.202 sec	total: 1.78s	remaining: 9m 14s
8:	learn 0.6939487229	test 0.6925235743	bestTest 0.703973956	passed: 0.192 sec	total: 1.97s	remainin

75:	learn 0.7121554668	test 0.7101481814	bestTest 0.7102604401	passed: 0.209 sec	total: 16.8s	remaining: 8m 56s
76:	learn 0.7127326952	test 0.7116075438	bestTest 0.7116075438	passed: 0.195 sec	total: 17s	remaining: 8m 55s
77:	learn 0.7129491558	test 0.7117198024	bestTest 0.7117198024	passed: 0.218 sec	total: 17.2s	remaining: 8m 54s
78:	learn 0.7134061282	test 0.7118320611	bestTest 0.7118320611	passed: 0.253 sec	total: 17.5s	remaining: 8m 55s
79:	learn 0.7137428448	test 0.7128423889	bestTest 0.7128423889	passed: 0.214 sec	total: 17.7s	remaining: 8m 55s
80:	learn 0.7143200731	test 0.7125056129	bestTest 0.7128423889	passed: 0.198 sec	total: 17.9s	remaining: 8m 54s
81:	learn 0.7147529944	test 0.7127301302	bestTest 0.7128423889	passed: 0.198 sec	total: 18.1s	remaining: 8m 53s
82:	learn 0.7153061715	test 0.7125056129	bestTest 0.7128423889	passed: 0.188 sec	total: 18.3s	remaining: 8m 52s
83:	learn 0.7153783251	test 0.7123933543	bestTest 0.7128423889	passed: 0.204 sec	total: 18.5s	remaining: 8

148:	learn 0.7243734667	test 0.7201392007	bestTest 0.7204759767	passed: 0.275 sec	total: 32.2s	remaining: 8m 27s
149:	learn 0.7244215691	test 0.7201392007	bestTest 0.7204759767	passed: 0.293 sec	total: 32.5s	remaining: 8m 28s
150:	learn 0.7244937226	test 0.7207004939	bestTest 0.7207004939	passed: 0.304 sec	total: 32.8s	remaining: 8m 29s
151:	learn 0.7246620809	test 0.7210372699	bestTest 0.7210372699	passed: 0.28 sec	total: 33s	remaining: 8m 30s
152:	learn 0.7246139785	test 0.7210372699	bestTest 0.7210372699	passed: 0.268 sec	total: 33.3s	remaining: 8m 31s
153:	learn 0.7246139785	test 0.7210372699	bestTest 0.7210372699	passed: 0.21 sec	total: 33.5s	remaining: 8m 30s
154:	learn 0.7247823368	test 0.7211495285	bestTest 0.7211495285	passed: 0.242 sec	total: 33.8s	remaining: 8m 30s
155:	learn 0.7245177738	test 0.7209250112	bestTest 0.7211495285	passed: 0.213 sec	total: 34s	remaining: 8m 30s
156:	learn 0.7245658762	test 0.7210372699	bestTest 0.7211495285	passed: 0.209 sec	total: 34.2s	remaini

221:	learn 0.7319736399	test 0.7272114953	bestTest 0.7272114953	passed: 0.205 sec	total: 48.5s	remaining: 8m 17s
222:	learn 0.7321660494	test 0.7270992366	bestTest 0.7272114953	passed: 0.185 sec	total: 48.7s	remaining: 8m 17s
223:	learn 0.7321660494	test 0.7268747194	bestTest 0.7272114953	passed: 0.218 sec	total: 48.9s	remaining: 8m 17s
224:	learn 0.7321419982	test 0.7270992366	bestTest 0.7272114953	passed: 0.192 sec	total: 49.1s	remaining: 8m 16s
225:	learn 0.732117947	test 0.7268747194	bestTest 0.7272114953	passed: 0.196 sec	total: 49.3s	remaining: 8m 16s
226:	learn 0.7323344076	test 0.726986978	bestTest 0.7272114953	passed: 0.19 sec	total: 49.5s	remaining: 8m 15s
227:	learn 0.7322863053	test 0.7267624607	bestTest 0.7272114953	passed: 0.195 sec	total: 49.7s	remaining: 8m 15s
228:	learn 0.732117947	test 0.7265379434	bestTest 0.7272114953	passed: 0.183 sec	total: 49.9s	remaining: 8m 14s
229:	learn 0.7322382029	test 0.7266502021	bestTest 0.7272114953	passed: 0.192 sec	total: 50.1s	remai

296:	learn 0.7363509548	test 0.7318140997	bestTest 0.7318140997	passed: 0.319 sec	total: 1m 5s	remaining: 8m 3s
297:	learn 0.7364471596	test 0.7313650651	bestTest 0.7318140997	passed: 0.192 sec	total: 1m 5s	remaining: 8m 3s
298:	learn 0.7366155178	test 0.7315895824	bestTest 0.7318140997	passed: 0.203 sec	total: 1m 5s	remaining: 8m 2s
299:	learn 0.7365674155	test 0.7313650651	bestTest 0.7318140997	passed: 0.191 sec	total: 1m 5s	remaining: 8m 2s
300:	learn 0.7366876714	test 0.7311405478	bestTest 0.7318140997	passed: 0.285 sec	total: 1m 6s	remaining: 8m 2s
301:	learn 0.736639569	test 0.7312528065	bestTest 0.7318140997	passed: 0.194 sec	total: 1m 6s	remaining: 8m 2s
302:	learn 0.7367117225	test 0.7310282892	bestTest 0.7318140997	passed: 0.203 sec	total: 1m 6s	remaining: 8m 1s
303:	learn 0.736639569	test 0.7306915132	bestTest 0.7318140997	passed: 0.276 sec	total: 1m 6s	remaining: 8m 2s
304:	learn 0.7368079273	test 0.7312528065	bestTest 0.7318140997	passed: 0.271 sec	total: 1m 7s	remaining: 

...

1466:	learn 0.7665351869	test 0.7561742254	bestTest 0.7567355186	passed: 0.301 sec	total: 5m 18s	remaining: 3m 44s
1467:	learn 0.7666073404	test 0.7560619668	bestTest 0.7567355186	passed: 0.296 sec	total: 5m 18s	remaining: 3m 44s
1468:	learn 0.766679494	test 0.7560619668	bestTest 0.7567355186	passed: 0.273 sec	total: 5m 19s	remaining: 3m 44s
1469:	learn 0.7666554428	test 0.7560619668	bestTest 0.7567355186	passed: 0.301 sec	total: 5m 19s	remaining: 3m 43s
1470:	learn 0.7666073404	test 0.7559497081	bestTest 0.7567355186	passed: 0.294 sec	total: 5m 19s	remaining: 3m 43s
1471:	learn 0.7665351869	test 0.7558374495	bestTest 0.7567355186	passed: 0.303 sec	total: 5m 20s	remaining: 3m 43s
1472:	learn 0.7666313916	test 0.7557251908	bestTest 0.7567355186	passed: 0.186 sec	total: 5m 20s	remaining: 3m 43s
1473:	learn 0.7666554428	test 0.7557251908	bestTest 0.7567355186	passed: 0.198 sec	total: 5m 20s	remaining: 3m 43s
1474:	learn 0.7666554428	test 0.7557251908	bestTest 0.7567355186	passed: 0.182 se

1538:	learn 0.7677858483	test 0.7562864841	bestTest 0.7567355186	passed: 0.203 sec	total: 5m 33s	remaining: 3m 28s
1539:	learn 0.7677617971	test 0.7562864841	bestTest 0.7567355186	passed: 0.183 sec	total: 5m 33s	remaining: 3m 27s
1540:	learn 0.7677617971	test 0.7562864841	bestTest 0.7567355186	passed: 0.204 sec	total: 5m 33s	remaining: 3m 27s
1541:	learn 0.7678098995	test 0.7565110013	bestTest 0.7567355186	passed: 0.192 sec	total: 5m 33s	remaining: 3m 27s
1542:	learn 0.7677617971	test 0.75662326	bestTest 0.7567355186	passed: 0.192 sec	total: 5m 34s	remaining: 3m 27s
1543:	learn 0.7677617971	test 0.75662326	bestTest 0.7567355186	passed: 0.192 sec	total: 5m 34s	remaining: 3m 26s
1544:	learn 0.7678098995	test 0.75662326	bestTest 0.7567355186	passed: 0.196 sec	total: 5m 34s	remaining: 3m 26s
1545:	learn 0.7678098995	test 0.75662326	bestTest 0.7567355186	passed: 0.187 sec	total: 5m 34s	remaining: 3m 26s
1546:	learn 0.767882053	test 0.75662326	bestTest 0.7567355186	passed: 0.193 sec	total: 5

1610:	learn 0.7688922026	test 0.7565110013	bestTest 0.7568477773	passed: 0.211 sec	total: 5m 49s	remaining: 3m 12s
1611:	learn 0.7688200491	test 0.7562864841	bestTest 0.7568477773	passed: 0.274 sec	total: 5m 49s	remaining: 3m 12s
1612:	learn 0.7688200491	test 0.7562864841	bestTest 0.7568477773	passed: 0.28 sec	total: 5m 49s	remaining: 3m 12s
1613:	learn 0.7688922026	test 0.7563987427	bestTest 0.7568477773	passed: 0.283 sec	total: 5m 50s	remaining: 3m 12s
1614:	learn 0.7688922026	test 0.7563987427	bestTest 0.7568477773	passed: 0.215 sec	total: 5m 50s	remaining: 3m 12s
1615:	learn 0.7688681514	test 0.7563987427	bestTest 0.7568477773	passed: 0.284 sec	total: 5m 50s	remaining: 3m 11s
1616:	learn 0.7688200491	test 0.7563987427	bestTest 0.7568477773	passed: 0.293 sec	total: 5m 51s	remaining: 3m 11s
1617:	learn 0.7687959979	test 0.7563987427	bestTest 0.7568477773	passed: 0.326 sec	total: 5m 51s	remaining: 3m 11s
1618:	learn 0.7687478955	test 0.7565110013	bestTest 0.7568477773	passed: 0.286 se

1683:	learn 0.7695656357	test 0.7568477773	bestTest 0.7569600359	passed: 0.285 sec	total: 6m 5s	remaining: 2m 57s
1684:	learn 0.7695175333	test 0.7568477773	bestTest 0.7569600359	passed: 0.274 sec	total: 6m 5s	remaining: 2m 57s
1685:	learn 0.7694934821	test 0.7569600359	bestTest 0.7569600359	passed: 0.288 sec	total: 6m 6s	remaining: 2m 56s
1686:	learn 0.7694694309	test 0.7569600359	bestTest 0.7569600359	passed: 0.291 sec	total: 6m 6s	remaining: 2m 56s
1687:	learn 0.7694213286	test 0.7570722946	bestTest 0.7570722946	passed: 0.181 sec	total: 6m 6s	remaining: 2m 56s
1688:	learn 0.7694213286	test 0.7570722946	bestTest 0.7570722946	passed: 0.197 sec	total: 6m 6s	remaining: 2m 56s
1689:	learn 0.7693732262	test 0.7570722946	bestTest 0.7570722946	passed: 0.183 sec	total: 6m 7s	remaining: 2m 55s
1690:	learn 0.7694213286	test 0.7570722946	bestTest 0.7570722946	passed: 0.222 sec	total: 6m 7s	remaining: 2m 55s
1691:	learn 0.7693251239	test 0.7571845532	bestTest 0.7571845532	passed: 0.265 sec	total

KeyboardInterrupt: 

In [ ]:
clf.score(xcv, ycv)

In [ ]:
clf.score(xva, yva)